## Install Dependencies 

In [ ]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git

In [ ]:
!pip install huggingface-hub

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install pretrained models

In [ ]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
!ls -lh {HOME}/weights

## Load Model

In [ ]:
from ultralytics import YOLOv10

In [ ]:
model = YOLOv10("/Users/alexis/Programmation/label-studio-ml-backend/yolo_backend/best.pt")

In [ ]:
model.train(data='dataset.yaml', epochs=500, batch=16, imgsz=640)

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/test/images/0a4299fb-frame_257.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/test/images/fafb4c4e-frame_40.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_8.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_72.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_116.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_323.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_433.jpg")

# Display the results
results[0].show()

In [12]:

import cv2
import numpy as np
from ultralytics import YOLOv10

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

track_history = defaultdict(lambda: [])
model = YOLOv10("/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train2_final/weights/best.pt")

# Perform object detection on an image
results = model("/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/images/0a2d048d-frame_444.jpg")

# Display the results
results[0].boxes


image 1/1 /Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/images/0a2d048d-frame_444.jpg: 480x640 1 fuel port, 305.8ms
Speed: 3.2ms preprocess, 305.8ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9747])
data: tensor([[309.6078, 178.4059, 390.5472, 252.7588,   0.9747,   0.0000]])
id: None
is_track: False
orig_shape: (480, 640)
shape: torch.Size([1, 6])
xywh: tensor([[350.0775, 215.5823,  80.9394,  74.3529]])
xywhn: tensor([[0.5470, 0.4491, 0.1265, 0.1549]])
xyxy: tensor([[309.6078, 178.4059, 390.5472, 252.7588]])
xyxyn: tensor([[0.4838, 0.3717, 0.6102, 0.5266]])

# Model Test

In [10]:
# Code took from the ultralytics repository

import cv2
import numpy as np
from ultralytics import YOLOv10

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

track_history = defaultdict(lambda: [])
model = YOLOv10("/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train2_final/weights/best.pt")
names = model.model.names

video_path = "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/video_lab_semiopen_1______3.avi"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter("video_lab_semiopen_1______3_object_tracking.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):
                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                #points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                #cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                #cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

result.release()
cap.release()
cv2.destroyAllWindows()

In [11]:
import torch
from ultralytics import YOLOv10


model_path = "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train14/weights/best_yolov10m.pt"
model = YOLOv10(model_path)

# Summary
model.info(detailed=True)

layer                                     name  gradient   parameters                shape         mu      sigma
    0                      model.0.conv.weight     False         1296        [48, 3, 3, 3]   -0.00278      0.151 torch.float32
    1                        model.0.bn.weight     False           48                 [48]       3.16      0.845 torch.float32
    2                          model.0.bn.bias     False           48                 [48]      0.278       1.49 torch.float32
    3                      model.1.conv.weight     False        41472       [96, 48, 3, 3]   -0.00102     0.0328 torch.float32
    4                        model.1.bn.weight     False           96                 [96]       2.05      0.572 torch.float32
    5                          model.1.bn.bias     False           96                 [96]      0.481      0.956 torch.float32
    6                  model.2.cv1.conv.weight     False         9216       [96, 96, 1, 1]   -0.00671     0.0526 torch.float3

(498, 16487602, 0, 63.9815424)

In [17]:
import torch
from ultralytics import YOLOv10


model_path = "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train15/weights/best_yolov10s.pt"
model = YOLOv10(model_path)

# Summary
model.info(detailed=True)

layer                                     name  gradient   parameters                shape         mu      sigma
    0                      model.0.conv.weight     False          864        [32, 3, 3, 3]   -0.00266      0.173 torch.float32
    1                        model.0.bn.weight     False           32                 [32]       3.16      0.727 torch.float32
    2                          model.0.bn.bias     False           32                 [32]    -0.0146       2.01 torch.float32
    3                      model.1.conv.weight     False        18432       [64, 32, 3, 3]   -0.00148     0.0465 torch.float32
    4                        model.1.bn.weight     False           64                 [64]       2.23      0.503 torch.float32
    5                          model.1.bn.bias     False           64                 [64]      0.538      0.955 torch.float32
    6                  model.2.cv1.conv.weight     False         4096       [64, 64, 1, 1]     -0.012     0.0767 torch.float3

(402, 8068674, 0, 24.7796736)

In [22]:
import torch
import torch.nn as nn

class YOLOFeatureExtractor(nn.Module):
    def __init__(self, yolo_model):
        super(YOLOFeatureExtractor, self).__init__()
        self.yolo = yolo_model
        # Selecting layer 6 for feature extraction
        self.feature_layer_index = 63

    def forward(self, x):
        features = None
        for i, layer in enumerate(self.yolo.model):
            x = layer(x)
            if i == self.feature_layer_index:
                features = x
                break
        return features, x  # Returning intermediate features and final output

# Assuming yolo_model is your pre-trained YOLO model
feature_extractor = YOLOFeatureExtractor(model)

In [ ]:
feature_extractor()

In [9]:
from ultralytics import YOLOv10

model = YOLOv10("/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train14/weights/best_yolov10m.pt")

# Preprocess and read image
image_path = "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/all/images/0a36f938-frame_2.jpg"

output = model.predict(image_path,--)


image 1/1 /Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/all/images/0a36f938-frame_2.jpg: 480x640 1 Fuel Port [SEMI-OPEN], 229.3ms
Speed: 1.3ms preprocess, 229.3ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)


In [1]:
from ultralytics.utils.benchmarks import benchmark
from ultralytics import YOLOv10
import torch

model_s = YOLOv10(
    "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train15/weights/best_yolov10s.pt"
)
model_m = YOLOv10(
    "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train14/weights/best_yolov10m.pt"
)
model_n = YOLOv10(
    "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train13/weights/best_yolov10n.pt"
)

In [2]:
model_n.val(
    data="/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/yolodataset.yaml",
    verbose=True,
)
model_s.val(
    data="/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/yolodataset.yaml",
    verbose=True,
)
model_m.val(
    data="/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/yolodataset.yaml",
    verbose=True,
)

Ultralytics YOLOv8.1.34 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
YOLOv10n summary (fused): 285 layers, 2695586 parameters, 0 gradients, 8.2 GFLOPs


val: Scanning /Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/balanced_dataset/test/labels... 1900 images, 703 backgrounds, 0 corrupt: 100%|██████████| 2574/2574 [00:01<00:00, 2514.42it/s]

val: New cache created: /Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/balanced_dataset/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 161/161 [10:14<00:00,  3.82s/it]


                   all       2574       1871       0.73      0.925      0.706      0.692
    Fuel Port [CLOSED]       2574        658      0.706      0.801      0.604      0.601
 Fuel Port [SEMI-OPEN]       2574        533      0.766       0.99      0.786      0.778
      Fuel Port [OPEN]       2574        680      0.717      0.984      0.726      0.697
Speed: 0.4ms preprocess, 234.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val17
Ultralytics YOLOv8.1.34 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
YOLOv10s summary (fused): 293 layers, 8037282 parameters, 0 gradients, 24.5 GFLOPs


val: Scanning /Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/balanced_dataset/test/labels.cache... 1900 images, 703 backgrounds, 0 corrupt: 100%|██████████| 2574/2574 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 161/161 [20:16<00:00,  7.56s/it]


                   all       2574       1871      0.725       0.92      0.692      0.679
    Fuel Port [CLOSED]       2574        658      0.703      0.798      0.585      0.582
 Fuel Port [SEMI-OPEN]       2574        533      0.759      0.974      0.781      0.774
      Fuel Port [OPEN]       2574        680      0.713      0.987      0.711      0.683
Speed: 0.4ms preprocess, 468.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val18
Ultralytics YOLOv8.1.34 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
YOLOv10m summary (fused): 369 layers, 16453858 parameters, 0 gradients, 63.4 GFLOPs


val: Scanning /Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/balanced_dataset/test/labels.cache... 1900 images, 703 backgrounds, 0 corrupt: 100%|██████████| 2574/2574 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 161/161 [33:37<00:00, 12.53s/it]


                   all       2574       1871      0.726      0.927      0.693       0.68
    Fuel Port [CLOSED]       2574        658      0.698      0.812      0.599      0.595
 Fuel Port [SEMI-OPEN]       2574        533      0.771      0.985      0.769      0.763
      Fuel Port [OPEN]       2574        680      0.711      0.985      0.712      0.682
Speed: 0.5ms preprocess, 779.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val19


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x31484e210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [3]:
from ultralytics.models.yolo.detect import DetectionPredictor
import torch
from ultralytics.utils import ops
from ultralytics.engine.results import Results


class YOLOv10DetectionPredictor(DetectionPredictor):
    def __init__(self, model):
        self.model = model
        
    def postprocess(self, preds, img, orig_imgs):
        if isinstance(preds, dict):
            preds = preds["one2one"]

        if isinstance(preds, (list, tuple)):
            preds = preds[0]

        if preds.shape[-1] == 6:
            pass
        else:
            preds = preds.transpose(-1, -2)
            bboxes, scores, labels = ops.v10postprocess(preds, 1, preds.shape[-1]-4)
            bboxes = ops.xywh2xyxy(bboxes)
            preds = torch.cat([bboxes, scores.unsqueeze(-1), labels.unsqueeze(-1)], dim=-1)

        mask = preds[..., 4] > 0.25
        preds = [p[mask[idx]] for idx, p in enumerate(preds)]

        if not isinstance(orig_imgs, list):  # input images are a torch.Tensor, not a list
            orig_imgs = ops.convert_torch2numpy_batch(orig_imgs)

        results = []
        for i, pred in enumerate(preds):
            orig_img = orig_imgs[i]
            pred[:, :4] = ops.scale_boxes(img.shape[2:], pred[:, :4], orig_img.shape)
            img_path = ""
            results.append(Results(orig_img, path=img_path, names=self.model.names, boxes=pred))
        return results


In [81]:
model_s.info(detailed=True)

layer                                     name  gradient   parameters                shape         mu      sigma
    0                      model.0.conv.weight     False          864        [32, 3, 3, 3]   -0.00266      0.173 torch.float32
    1                        model.0.bn.weight     False           32                 [32]       3.16      0.727 torch.float32
    2                          model.0.bn.bias     False           32                 [32]    -0.0146       2.01 torch.float32
    3                      model.1.conv.weight     False        18432       [64, 32, 3, 3]   -0.00148     0.0465 torch.float32
    4                        model.1.bn.weight     False           64                 [64]       2.23      0.503 torch.float32
    5                          model.1.bn.bias     False           64                 [64]      0.538      0.955 torch.float32
    6                  model.2.cv1.conv.weight     False         4096       [64, 64, 1, 1]     -0.012     0.0767 torch.float3

(402, 8068674, 0, 24.7796736)

In [1]:
import torch
import cv2

# Load the YOLOv10 model
model = torch.load("yolov10s.pth")
model.eval()

# Global variable to store intermediate output
intermediate_output = None

# Function to capture the intermediate layer output
def get_intermediate_output_hook(module, input, output):
    global intermediate_output
    intermediate_output = output

# Register the hook on the desired layer
layer_name = '10'  # Example layer, change according to your requirement
for name, layer in model.model.named_modules():
    if name == layer_name:
        layer.register_forward_hook(get_intermediate_output_hook)
        break
else:
    raise ValueError(f"Layer {layer_name} not found in the model.")

# Load and preprocess the image
image_path = "/Users/alexis/Downloads/Picture 1.jpg"
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img / 255.0  # Normalize image to 0-1 range
img = torch.from_numpy(img).permute(2, 0, 1).float().unsqueeze(0)  # Preprocess image

# Run the model
with torch.no_grad():
    results = model.predict(img)

# The intermediate output is now stored in the global variable `intermediate_output`
feature_vector = intermediate_output.squeeze().detach().numpy()

yolov10predictor = YOLOv10DetectionPredictor(model)

# Postprocess the model output
results = yolov10predictor.postprocess(results, img, img)

# Display the image with the detected objects
results[0].show()

print("Feature Vector:", feature_vector.shape)

""" # Check and handle the model's output correctly
if isinstance(results, dict):
    if 'pred' in results:
        predictions = results['pred']
    else:
        raise ValueError("The 'pred' key is not found in the model's output dictionary.")
else:
    predictions = results[0] if isinstance(results, (list, tuple)) else results """

# Print or use both the feature vector and the predictions
#print("Feature Vector:", feature_vector)
#print("Predictions:", predictions) 

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 116 but got size 115 for tensor number 1 in the list.

In [1]:
import torch

test = torch.load("/Volumes/ALEXIS/Thesis/Features/22/0af969ec-frame_54.pt")

test.shape

(512, 20, 20)

In [49]:
predict = model_n.predict("/Users/alexis/Downloads/frame_0 copy 2.jpg")
predict[0].show()   


image 1/1 /Users/alexis/Downloads/frame_0 copy 2.jpg: 384x640 1 Fuel Port [CLOSED], 83.1ms
Speed: 2.1ms preprocess, 83.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)


In [53]:
predict = model_s("/Users/alexis/Downloads/frame_0.jpg", imgsz=(640, 640))    
predict[0].show()


image 1/1 /Users/alexis/Downloads/frame_0.jpg: 640x640 (no detections), 204.4ms
Speed: 5.3ms preprocess, 204.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
